In [1]:
import findspark
findspark.init("/opt/manual/spark")

In [2]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder \
.appName("Filter Data") \
.master("local[2]") \
.getOrCreate()

In [4]:
df = spark.read \
.option("header", True) \
.option("inferSchema", True) \
.option("compression", "gzip") \
.csv("file:///home/train/datasets/Hotel_Reviews.csv.gz")

In [5]:
df2 = df.withColumn("Tags",
                   F.split(F.col("Tags"),",")
                   .cast(ArrayType(StringType())))\
.withColumn("Review_Date", F.to_date("Review_Date","M/d/yyyy"))

In [6]:
df2.limit(2).toPandas()

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683


In [8]:
df2.filter("Average_Score < 6.0").count()

65

In [9]:
df2.filter("Reviewer_Nationality == 'Russia' ").limit(3).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng


In [10]:
df.filter("Reviewer_Nationality LIKE '%Russia%'").limit(3).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,4/20/2017,7.7,Hotel Arena,Russia,The glass wall separating the bathroom and th...,29,1403,Very nice hotel located in a park Ca 30 minut...,56,85,8.8,"[' Leisure trip ', ' Couple ', ' Duplex Double...",105 day,52.3605759,4.9159683
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,12/13/2015,7.7,Hotel Arena,Russia,Arena hotel is located in a historical buildi...,87,1403,Location is good it takes 20 mins by feet and...,23,6,6.0,"[' Solo traveler ', ' Duplex Double Room ', ' ...",599 day,52.3605759,4.9159683


In [11]:
df2.selectExpr("TRIM(Reviewer_Nationality) AS Reviewer_Nationality") \
.filter("Reviewer_Nationality == 'Russia'").limit(3).toPandas()

,Reviewer_Nationality
0,Russia
1,Russia
2,Russia


In [13]:
df2.filter(F.col("Hotel_Address").rlike("Milan")).limit(3).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,6 Via Aristotile Fioravanti Garibaldi Station ...,30,2017-08-01,8.9,Hotel VIU Milan,Netherlands,There is still some construction taking place...,50,219,Very friendly staff great service from everyo...,34,21,9.6,"[[' Leisure trip ', ' Couple ', ' Superior D...",2 days,45.488764,9.183057
1,6 Via Aristotile Fioravanti Garibaldi Station ...,30,2017-07-14,8.9,Hotel VIU Milan,Russia,The hotel blocked double 1 night payment on m...,149,219,No Positive,0,5,5.4,"[[' Leisure trip ', ' Couple ', ' Superior D...",20 days,45.488764,9.183057
2,6 Via Aristotile Fioravanti Garibaldi Station ...,30,2017-07-12,8.9,Hotel VIU Milan,Belgium,Not the best neighborhood in Milan right next...,20,219,Brand new and beautifully designed hotel bar ...,13,25,8.8,"[[' Business trip ', ' Solo traveler ', ' Su...",22 days,45.488764,9.183057
